In [39]:
import pandas as pd
import datetime as dt
from tqdm import tqdm

print("Vaccinnation data")
df = pd.read_csv ("../data/train/vaccination/fr/vaccination_hist_data.csv", sep =";")
df['departement'] = df['departement'].replace({'2A':'201','2B':'202'}).astype(int)
df = df[df['departement']<203]
df["date_debut_semaine"]=pd.to_datetime(df["date_debut_semaine"], dayfirst = True) 
df2 = pd.read_csv("../data/train/all_data_merged/fr/Enriched_Covid_history_data.csv", sep = ",")
print(df)
print(df2)
dfvac1 = df[df["rang_vaccinal"]==1].reset_index()
dfvac2 = df[df["rang_vaccinal"]==2].reset_index()
print(dfvac1)
print(dfvac2)
dfvac1list = []
dfvac2list = []
referencedate = "2021-01-18"
referencedate=pd.to_datetime(referencedate)
referencedate2 = "2021-01-25"
referencedate2=pd.to_datetime(referencedate2)
df2["time"]=pd.to_datetime(df2["time"])
#df['departement'] = df['departement'].replace({'2A':'201','2B':'202'}).astype(int)


cumvac1list = []
cumvac2list = []
cum1 = dfvac1.groupby(['departement', 'date_debut_semaine']).sum().groupby(level=0).cumsum().sort_values("date_debut_semaine").reset_index().drop(columns = "index")
cum2 = dfvac2.groupby(['departement', 'date_debut_semaine']).sum().groupby(level=0).cumsum().sort_values("date_debut_semaine").reset_index().drop(columns = "index")

print(cum1)
print(cum2)

for i in tqdm(df2.index):
    date = df2.loc[i,"time"]
    depnum = df2.loc[i,"numero"]
    datefound = False
    datefound2 = False
    for j in cum1.index:
        counter = 0
        if cum1.loc[j,"departement"]==depnum: 
            date1 = cum1.loc[j,"date_debut_semaine"]
            if (date < referencedate):
                dfvac1list.append((date,0))
                counter +=1
                datefound = True

            elif ((date >= referencedate) & (date1 <= date) & (date < (date1 + pd.Timedelta("7 days")))):
                cumvac1 = cum1.loc[j,"nb"]
                dfvac1list.append((date,cumvac1))
                counter += 1
                datefound = True
            if (counter == 1):
                break
    if datefound == False:
        dfvac1list.append((date, cumvac1))

    for k in cum2.index:
        counter = 0
        if cum2.loc[k,"departement"]==depnum: 
            date2 = cum2.loc[k,"date_debut_semaine"]
            if (date < referencedate2):
                dfvac2list.append((date,0))
                counter +=1
                datefound2 = True
            elif ((date >= referencedate2) & (date2 <= date) & (date < (date2 + pd.Timedelta("7 days")))):
                cumvac2 = cum2.loc[k,"nb"]
                dfvac2list.append((date, cumvac2))
                counter +=1
                datefound2 = True
            if (counter == 1):
                break
    if datefound2 == False:
        dfvac2list.append((date, cumvac2))
                
            

dfvac1 = pd.DataFrame(dfvac1list)
dfvac1.columns=["date", "vac1nb"]
print(dfvac1)
dfvac2 = pd.DataFrame(dfvac2list)
dfvac2.columns=["date", "vac2nb"]
print(dfvac2)

df2["vac1nb"]=dfvac1["vac1nb"]
df2["vac2nb"]=dfvac2["vac2nb"]
df2.dropna(inplace = True)
df2.to_csv("../data/train/all_data_merged/fr/Enriched_Covid_history_data.csv", index = False)
print(df2)


Vaccinnation data


  0%|          | 0/30590 [00:00<?, ?it/s]

      code_region region  departement  rang_vaccinal date_debut_semaine     nb
0              84    ARA            1              1         2021-01-18   1694
1              84    ARA            1              1         2021-01-25   2014
2              84    ARA            1              2         2021-01-25    119
3              84    ARA            1              1         2021-02-01   2017
4              84    ARA            1              2         2021-02-01   1269
...           ...    ...          ...            ...                ...    ...
2845           11    IDF           95              2         2021-04-12  12744
2846           11    IDF           95              1         2021-04-19   1038
2847           11    IDF           95              2         2021-04-19  17540
2848           11    IDF           95              1         2021-04-26    448
2849           11    IDF           95              2         2021-04-26  12182

[2850 rows x 6 columns]
              nom  numero  

100%|██████████| 30590/30590 [04:54<00:00, 103.70it/s]


            date  vac1nb
0     2020-05-14       0
1     2020-05-15       0
2     2020-05-16       0
3     2020-05-17       0
4     2020-05-18       0
...          ...     ...
30585 2021-03-27   71799
30586 2021-03-28   71799
30587 2021-03-29   85369
30588 2021-03-30   85369
30589 2021-03-31   85369

[30590 rows x 2 columns]
            date  vac2nb
0     2020-05-14       0
1     2020-05-15       0
2     2020-05-16       0
3     2020-05-17       0
4     2020-05-18       0
...          ...     ...
30585 2021-03-27   23991
30586 2021-03-28   23991
30587 2021-03-29   34017
30588 2021-03-30   34017
30589 2021-03-31   34017

[30590 rows x 2 columns]
              nom  numero       time  hospi  reanim  newhospi  newreanim  \
1             Ain     1.0 2020-05-15  135.0     7.0       4.0        0.0   
2             Ain     1.0 2020-05-16  134.0     6.0       1.0        0.0   
3             Ain     1.0 2020-05-17  133.0     6.0       1.0        0.0   
4             Ain     1.0 2020-05-18  132.0 